In [11]:
!pip --version

pip 20.0.2 from /usr/lib/python3/dist-packages/pip (python 3.8)


In [12]:
!pip install trivial-torch-tools

In [13]:
import torch.nn.functional as F
import torch, torchvision
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [14]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.FashionMNIST('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.FashionMNIST('./files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [15]:
from trivial_torch_tools import Sequential
from collections import OrderedDict
globals()["default_device"] = torch.device("cpu")
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        layers = Sequential(input_shape = (1,28,28)) # given the input shape size
        layers.add_module('conv1', nn.Conv2d(1, 10, kernel_size=5))
        layers.add_module('conv1_pool', nn.MaxPool2d(2))
        layers.add_module('relu', nn.ReLU())
        layers.add_module('conv2', nn.Conv2d(10, 10, kernel_size=5))
        layers.add_module('conv2_pool', nn.MaxPool2d(2))
        layers.add_module('relu2', nn.ReLU())
        layers.add_module("conv2_drop", nn.Dropout2d())
        layers.add_module('flatten', nn.Flatten(1)) 
        layers.add_module('fc1', nn.Linear(layers.output_size, 50))
        layers.add_module('relu3', nn.ReLU())
        layers.add_module('fc2', nn.Linear(layers.output_size, 10)) 
        layers.add_module('softmax', nn.LogSoftmax())
        self.layers = layers

    def forward(self, x):
        return self.layers(x)

In [16]:

network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [17]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [18]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), './files/model.pth')
      torch.save(optimizer.state_dict(), './files/optimizer.pth')

In [19]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [20]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

NameError: name 'functools' is not defined